In [ ]:
!pip install gmdh

In [ ]:
import pandas as pd
import numpy as np
try:
    df = pd.read_csv("pokemon.csv", sep=",", encoding='utf-8')
except UnicodeDecodeError:
    df = pd.read_csv("pokemon.csv", sep=",", encoding='latin1')

print(df.head())
data = df.drop(columns=['Name'])
print(data.head())
print(data.info())
print(data.describe())

         Name Type 1  Type 2  Combat Power  HP  Attack  Defense  Sp. Atk  \
0   Bulbasaur  Grass  Poison           318  45      49       49       65   
1     Ivysaur  Grass  Poison           405  60      62       63       80   
2    Venusaur  Grass  Poison           525  80      82       83      100   
3  Charmander   Fire     NaN           309  39      52       43       60   
4  Charmeleon   Fire     NaN           405  58      64       58       80   

   Sp. Def  Speed  Generation  Legendary  
0       65     45           1      False  
1       80     60           1      False  
2      100     80           1      False  
3       50     65           1      False  
4       65     80           1      False  
  Type 1  Type 2  Combat Power  HP  Attack  Defense  Sp. Atk  Sp. Def  Speed  \
0  Grass  Poison           318  45      49       49       65       65     45   
1  Grass  Poison           405  60      62       63       80       80     60   
2  Grass  Poison           525  80      82   

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [ ]:
type_1_mode = data['Type 1'].mode()[0]
data['Type 1'].fillna(type_1_mode, inplace=True)

# Обработка категории Type 2
type_2_mode = data['Type 2'].mode()[0]
data['Type 2'].fillna(type_2_mode, inplace=True)

# Преобразование категориальных данных в числовую форму
unique_types = set(data['Type 1']).union(set(data['Type 2'].dropna()))
type_dict = {type_name: index for index, type_name in enumerate(unique_types)}

data['Type 1'] = data['Type 1'].map(type_dict)
data['Type 2'] = data['Type 2'].map(lambda x: type_dict[x] if pd.notna(x) else type_dict[type_2_mode])

# Обработка столбца Legendary
data['Legendary'] = data['Legendary'].astype(int)
print(data.info())
print(data.head())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 151 entries, 0 to 150
Data columns (total 11 columns):
 #   Column        Non-Null Count  Dtype
---  ------        --------------  -----
 0   Type 1        151 non-null    int64
 1   Type 2        151 non-null    int64
 2   Combat Power  151 non-null    int64
 3   HP            151 non-null    int64
 4   Attack        151 non-null    int64
 5   Defense       151 non-null    int64
 6   Sp. Atk       151 non-null    int64
 7   Sp. Def       151 non-null    int64
 8   Speed         151 non-null    int64
 9   Generation    151 non-null    int64
 10  Legendary     151 non-null    int64
dtypes: int64(11)
memory usage: 13.1 KB
None
   Type 1  Type 2  Combat Power  HP  Attack  Defense  Sp. Atk  Sp. Def  Speed  \
0       2       9           318  45      49       49       65       65     45   
1       2       9           405  60      62       63       80       80     60   
2       2       9           525  80      82       83      100      100    

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [ ]:
from sklearn.model_selection import train_test_split
from gmdh import split_data
X = data.drop(['Combat Power'], axis=1)
y = data['Combat Power']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
gmdh_x_train, gmdh_x_test, gmdh_y_train, gmdh_y_test = gmdh.split_data(X, y, test_size=0.2, random_state=42)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


#Стекинг модель

In [ ]:
from sklearn.ensemble import StackingRegressor
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.svm import SVR

estimators = [
    ('rf', RandomForestRegressor(n_estimators=10, random_state=42)),
    ('svr', SVR())
]

stacking_reg = StackingRegressor(estimators=estimators, final_estimator=LinearRegression())

stacking_reg.fit(X_train, y_train)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


StackingRegressor(estimators=[('rf',
                               RandomForestRegressor(n_estimators=10,
                                                     random_state=42)),
                              ('svr', SVR())],
                  final_estimator=LinearRegression())

#Модель многослойного персептрона

In [ ]:
from sklearn.neural_network import MLPRegressor

mlp_reg = MLPRegressor(hidden_layer_sizes=(100, ), max_iter=300, alpha=0.0001, random_state=42)


mlp_reg.fit(X_train, y_train)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


MLPRegressor(max_iter=300, random_state=42)

#Методы МГУА

In [ ]:
import gmdh

if X_train.shape[1] == 0:
    print("X_train is empty")
if y_train.size == 0:
    print("y_train is empty")
y_train = np.array(y_train)
print(X_train, y_train)
    # Модель Combi
comb1_reg = gmdh.Combi()
comb1_reg.fit(gmdh_x_train, gmdh_y_train)

    # Модель Mia
m1a_reg = gmdh.Mia()
m1a_reg.fit(gmdh_x_train, gmdh_y_train)

<class 'pandas.core.frame.DataFrame'> [251 495 395 490 290 405 450 455 405 365 320 580 680 349 205 490 500 495
 435 305 355 340 450 318 385 390 275 245 490 300 525 500 320 485 438 325
 435 420 520 305 345 495 300 505 385 425 310 475 328 320 534 483 495 265
 483 325 500 323 320 405 395 490 325 455 310 525 395 305 530 205 500 200
 309 479 270 515 288 314 295 480 525 400 500 385 405 450 405 340 510 600
 490 300 310 455 555 525 450 500 442 350 580 540 505 515 405 440 535 320
 330 440 325 390 460 600 262 335 455 395 405 520]


/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


ValueError: The truth value of a Series is ambiguous. Use a.empty, a.bool(), a.item(), a.any() or a.all().

In [ ]:
from sklearn.metrics import mean_squared_error, r2_score

# Прогнозы модели нпя
stacking_pred = stacking_reg.predict(X_test)
mlp_pred = mlp_reg.predict(X_test)
linear_gmdh_pred = comb1_reg.predict(X_test)
nonlinear_gmdh_pred = m1a_reg.predict(X_test)

# Расчет метрик
stacking_mse = mean_squared_error(y_test, stacking_pred)
mlp_mse = mean_squared_error(y_test, mlp_pred)
linear_gmdh_mse = mean_squared_error(y_test, linear_gmdh_pred)
nonlinear_gmdh_mse = mean_squared_error(y_test, nonlinear_gmdh_pred)

stacking_r2 = r2_score(y_test, stacking_pred)
mlp_r2 = r2_score(y_test, mlp_pred)
linear_gmdh_r2 = r2_score(y_test, linear_gmdh_pred)
nonlinear_gmdh_r2 = r2_score(y_test, nonlinear_gmdh_pred)

print(f"Stacking Regressor MSE: {stacking_mse}, R2: {stacking_r2}")
print(f"MLP Regressor MSE: {mlp_mse}, R2: {mlp_r2}")
print(f"Linear GMDH Regressor MSE: {linear_gmdh_mse}, R2: {linear_gmdh_r2}")
print(f"Nonlinear GMDH Regressor MSE: {nonlinear_gmdh_mse}, R2: {nonlinear_gmdh_r2}")


/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


ValueError: Matrix 'X' must have 0 columns because there were 0 columns in the training 'X' matrix